## ImageSynthesizer Example

This notebook demonstrates how to use the `ImageSynthesizer` for generating image-based test cases:
- Uses image generation models (Gemini Imagen, DALL-E, etc.) to create images
- Creates test cases for evaluating generated images with `ImageJudge`
- Stores images directly in the database for persistence
- Provides rich metadata including generation prompts and expected outputs

The `ImageSynthesizer` is designed for:
- **Image Generation Testing**: Generate images from text prompts and evaluate quality
- **Visual Content Validation**: Create test sets for image generation models
- **Automated Visual QA**: Build pipelines for testing image generation capabilities

Key Configuration Parameters:
- `prompt`: Text description for image generation
- `model`: Image generation model (e.g., "gemini/imagen-3.0-generate-002")
- `text_model`: Text model for generating test set properties (default: "gemini/gemini-2.0-flash")
- `expected_output_template`: Description of what generated images should contain
- `category`, `topic`, `behavior`: Classification for generated tests
- `image_size`: Dimensions of generated images (default: "1024x1024")
- `batch_size`: Maximum tests per generation batch (default: 5)

> **Note**: Imagen models only generate images - they cannot generate text. A separate `text_model` is used to auto-generate test set name and description.


### Setup: Configure API Credentials


In [2]:
# Set up your API credentials and configuration
import os

from rhesis.sdk.synthesizers import ImageSynthesizer
from rhesis.sdk.metrics import ImageJudge

# Configure your API credentials
# For Gemini Imagen:
#os.environ["GEMINI_API_KEY"] = "replace-with-your-gemini-api-key"

# For Rhesis backend (if pushing test sets):
#os.environ["RHESIS_API_KEY"] = "replace-with-your-rhesis-api-key"

print("✓ SDK configured successfully")
print("Ready to generate image test cases with ImageSynthesizer!")


/Users/harry/rhesis/Dev/rhesis/sdk/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ SDK configured successfully
Ready to generate image test cases with ImageSynthesizer!


## Example 1: Basic ImageSynthesizer Usage

Generate simple image tests with default settings.


In [ ]:
# Create a basic image synthesizer
# Note: Imagen models only generate images, so a separate text_model is used
# for generating test set properties (name, description). Defaults to gemini-2.0-flash.
synthesizer = ImageSynthesizer(
    prompt="A serene mountain landscape at sunset with orange and purple sky",
    model="gemini/imagen-3.0-generate-002",  # Image generation model
    # text_model defaults to "gemini/gemini-2.0-flash" for generating test set properties
    expected_output_template="Image should contain mountains with warm sunset colors (orange, purple, pink)"
)

# Generate image tests
result = synthesizer.generate(num_tests=3)

print(f"Generated {len(result.tests)} image tests")
print(f"Test set type: {result.test_set_type}")
print(f"Test set metadata keys: {list(result.metadata.keys())}")


Generating images: 100%|██████████| 3/3 [00:28<00:00,  9.55s/it]


NotFoundError: litellm.NotFoundError: VertexAIException - {
  "error": {
    "code": 404,
    "message": "models/imagen-4.0-generate-001 is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.",
    "status": "NOT_FOUND"
  }
}


In [ ]:
# Inspect the structure of generated tests
first_test = result.tests[0]

print("Test Structure:")
print(f"- Test Type: {first_test.test_type}")
print(f"- Behavior: {first_test.behavior}")
print(f"- Category: {first_test.category}")
print(f"- Topic: {first_test.topic}")
print(f"- Has binary data: {first_test.test_binary is not None}")
if first_test.test_binary:
    print(f"- Binary size: {len(first_test.test_binary)} bytes")

print(f"\nMetadata keys: {list(first_test.metadata.keys())}")
print(f"- MIME type: {first_test.metadata.get('binary_mime_type')}")
print(f"- Generation prompt: {first_test.metadata.get('generation_prompt')[:80]}...")
print(f"- Expected output: {first_test.metadata.get('expected_output')}")
print(f"- Model used: {first_test.metadata.get('model')}")


### Display Generated Images

View the generated images directly in the notebook.


In [ ]:
from IPython.display import display, Image as IPImage

# Display generated images
for i, test in enumerate(result.tests):
    if test.test_binary:
        print(f"\n=== Image {i+1} ===")
        print(f"Prompt: {test.metadata.get('generation_prompt', 'N/A')}")
        
        # Display the image
        display(IPImage(data=test.test_binary))
        print(f"Size: {len(test.test_binary)} bytes")


## Example 2: Evaluating Generated Images with ImageJudge

Use the `ImageJudge` metric to evaluate the quality of generated images.


In [ ]:
import tempfile

# Create an ImageJudge for evaluation
judge = ImageJudge(
    model="gemini/gemini-1.5-flash",  # Vision-capable model for evaluation
    categories=["pass", "partial", "fail"],
    passing_categories=["pass"]
)

print("=== Evaluating Generated Images ===")
print(f"Judge model: {judge.model.model_name}")
print(f"Categories: {judge.config.categories}")
print()

# Evaluate each generated image
for i, test in enumerate(result.tests):
    if test.test_binary:
        # Save image temporarily for evaluation
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as f:
            f.write(test.test_binary)
            temp_path = f.name
        
        try:
            # Evaluate the image
            evaluation = judge.evaluate(
                input=test.metadata.get('generation_prompt', ''),
                output=temp_path,
                expected_output=test.metadata.get('expected_output', '')
            )
            
            print(f"Image {i+1}:")
            print(f"  Score: {evaluation.score}")
            print(f"  Passed: {evaluation.is_successful}")
            print(f"  Reason: {evaluation.details.get('reason', 'N/A')[:150]}...")
            print()
        finally:
            import os
            os.unlink(temp_path)  # Clean up temp file


## Example 3: Custom Configuration

Generate image tests with custom categories, topics, and behaviors.


In [ ]:
# Create synthesizer with custom configuration
# You can also specify a custom text_model for property generation
product_synthesizer = ImageSynthesizer(
    prompt="Professional product photo of a modern smartphone on a clean white background",
    model="gemini/imagen-3.0-generate-002",  # Image generation
    text_model="gemini/gemini-2.0-flash",    # Text generation for properties (optional)
    expected_output_template="Clean product photo with smartphone centered, white background, professional lighting",
    category="E-Commerce",
    topic="Product Photography",
    behavior="Visual Quality",
    image_size="1024x1024"
)

product_result = product_synthesizer.generate(num_tests=2)

print("=== Product Photography Test Set ===")
print(f"Generated {len(product_result.tests)} tests")

for i, test in enumerate(product_result.tests):
    print(f"\nTest {i+1}:")
    print(f"  Category: {test.category}")
    print(f"  Topic: {test.topic}")
    print(f"  Behavior: {test.behavior}")
    print(f"  Image size: {test.metadata.get('image_size')}")


## Example 4: Pushing Test Sets to Backend

Save the generated image tests to the Rhesis backend for persistent storage.

# First, set up the test set properties
result.name = "Mountain Landscape Image Tests"
result.description = "Generated image tests for mountain landscape scenes"
result.short_description = "Mountain landscape images"

print("Test Set Properties:")
print(f"  Name: {result.name}")
print(f"  Description: {result.description}")
print(f"  Test count: {len(result.tests)}")
print(f"  Test type: {result.test_set_type}")

# Uncomment to push to backend:
# response = result.push()
# print(f"\nPushed test set with ID: {result.id}")

print("\n(Uncomment the push() call to save to backend)")

## Example 5: Saving Generated Images to Disk

Export the generated images to local files for inspection.


In [ ]:
from pathlib import Path

# Create output directory
output_dir = Path("generated_images")
output_dir.mkdir(exist_ok=True)

print(f"Saving images to: {output_dir.absolute()}")

# Save each generated image
saved_count = 0
for i, test in enumerate(result.tests):
    if test.test_binary:
        # Determine file extension from MIME type
        mime_type = test.metadata.get('binary_mime_type', 'image/png')
        ext = mime_type.split('/')[-1]
        if ext == 'jpeg':
            ext = 'jpg'
        
        # Save the image
        filename = output_dir / f"image_{i+1}.{ext}"
        with open(filename, 'wb') as f:
            f.write(test.test_binary)
        
        print(f"Saved: {filename} ({len(test.test_binary)} bytes)")
        saved_count += 1

print(f"\n✓ Saved {saved_count} images to {output_dir}")
